# Classificador CNN Simples para Emoções Faciais (Raiva vs Alegria)

## Objetivo Pedagógico

Este notebook implementa um classificador binário de emoções usando uma CNN (Convolutional Neural Network) simples.
O objetivo é comparar o desempenho de CNNs tradicionais com modelos foundation (como CLIP) em classificação de emoções.

### Arquitetura da CNN

A arquitetura utilizada é composta por:
- **3 blocos convolucionais**: Cada bloco contém uma camada Conv2D seguida de MaxPooling2D
  - Conv2D extrai características visuais (bordas, texturas, padrões)
  - MaxPooling2D reduz dimensionalidade e adiciona invariância espacial
- **Dropout (0.4)**: Regularização para prevenir overfitting
- **Camadas densas**: Classificação final baseada nas características extraídas

### Metodologia de Avaliação

- **30 simulações independentes**: Cada simulação usa um conjunto diferente de 50 imagens por classe
- **Métricas estatísticas**: Média e desvio padrão da acurácia permitem avaliar robustez
- **Split treino/validação**: 80% treino, 20% validação (dentro de cada simulação)

### Conceitos Importantes

**Por que usar múltiplas simulações?**
- Reduz viés de seleção de amostras específicas
- Permite calcular intervalos de confiança
- Avalia estabilidade do modelo em diferentes subconjuntos de dados

**Por que classificação binária?**
- Raiva e alegria são emoções opostas e bem distintas
- Facilita comparação com modelos foundation
- Permite focar na capacidade de discriminação das características

In [ ]:
# Importações necessárias
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
import json
from datetime import datetime

# Configuração para reprodutibilidade
# A seed garante que os mesmos resultados sejam obtidos em execuções diferentes
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Verifica disponibilidade de GPU
print("TensorFlow version:", tf.__version__)
print("GPU disponível:", tf.config.list_physical_devices('GPU'))
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

In [ ]:
# Configurações do projeto

# Caminhos
PROJECT_ROOT = Path("/projeto-estudo-comparativo")
DATASET_DIR = PROJECT_ROOT / "datasets"
RESULTS_DIR = PROJECT_ROOT / "results" / "simple_cnn"
MODELS_DIR = PROJECT_ROOT / "models" / "simple_cnn"

# Cria diretórios se não existirem
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
MODELS_DIR.mkdir(parents=True, exist_ok=True)

# Hiperparâmetros
# Explicação: Imagens redimensionadas para 128x128 pixels (balanço entre performance e qualidade)
IMG_WIDTH = 128
IMG_HEIGHT = 128

# Explicação: Split de validação - 20% dos dados para validação, 80% para treino
VALIDATION_SPLIT = 0.2

# Explicação: Batch size - número de imagens processadas simultaneamente
# Valores maiores = treinamento mais rápido mas requer mais memória
BATCH_SIZE = 32

# Explicação: Número máximo de épocas de treinamento
# Early stopping pode interromper antes se não houver melhoria
EPOCHS = 50

# Número de simulações
NUM_SIMULATIONS = 30

# Classes
CLASSES = ['alegria', 'raiva']
NUM_CLASSES = len(CLASSES)

print(f"Configuração carregada:")
print(f"  Dataset: {DATASET_DIR}")
print(f"  Resultados: {RESULTS_DIR}")
print(f"  Modelos: {MODELS_DIR}")
print(f"  Imagens: {IMG_WIDTH}x{IMG_HEIGHT}")
print(f"  Classes: {CLASSES}")
print(f"  Simulações: {NUM_SIMULATIONS}")

In [ ]:
def create_simple_cnn_model():
    """
    Cria um modelo CNN simples para classificação binária de emoções.
    
    Arquitetura detalhada:
    ----------------------
    1. Input Layer (128x128x3): Imagens RGB
    
    2. Rescaling Layer: Normaliza pixels de [0, 255] para [0, 1]
       - Facilita convergência do treinamento
       - Evita problemas de gradientes explodindo/desaparecendo
    
    3. Bloco Convolucional 1:
       - Conv2D(32 filtros, kernel 3x3): Detecta características básicas (bordas, cantos)
       - MaxPooling2D(2x2): Reduz dimensionalidade, mantém características importantes
       - Output: 64x64x32
    
    4. Bloco Convolucional 2:
       - Conv2D(64 filtros, kernel 3x3): Detecta padrões mais complexos (olhos, boca)
       - MaxPooling2D(2x2): Reduz dimensionalidade
       - Output: 32x32x64
    
    5. Bloco Convolucional 3:
       - Conv2D(128 filtros, kernel 3x3): Detecta características de alto nível (expressões)
       - MaxPooling2D(2x2): Reduz dimensionalidade
       - Output: 16x16x128
    
    6. Dropout(0.4): Desativa aleatoriamente 40% dos neurônios durante treino
       - Previne overfitting
       - Força rede a aprender características robustas
    
    7. Flatten: Converte tensor 3D em vetor 1D (16*16*128 = 32768 valores)
    
    8. Dense(64, relu): Camada totalmente conectada
       - Combina características extraídas
       - ReLU adiciona não-linearidade
    
    9. Dense(1, sigmoid): Camada de saída para classificação binária
       - Sigmoid produz probabilidade entre 0 e 1
       - 0 = raiva, 1 = alegria (ou vice-versa)
    
    Total de parâmetros: ~2.1M parâmetros treináveis
    
    Returns:
        modelo Keras compilado
    """
    model = keras.Sequential([
        # Input
        layers.Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3)),
        
        # Normalização
        layers.Rescaling(1./255),
        
        # Bloco 1: Características básicas
        layers.Conv2D(32, 3, padding='same', activation='relu', name='conv1'),
        layers.MaxPooling2D(2, name='pool1'),
        
        # Bloco 2: Características intermediárias
        layers.Conv2D(64, 3, padding='same', activation='relu', name='conv2'),
        layers.MaxPooling2D(2, name='pool2'),
        
        # Bloco 3: Características de alto nível
        layers.Conv2D(128, 3, padding='same', activation='relu', name='conv3'),
        layers.MaxPooling2D(2, name='pool3'),
        
        # Regularização
        layers.Dropout(0.4, name='dropout'),
        
        # Classificação
        layers.Flatten(name='flatten'),
        layers.Dense(64, activation='relu', name='dense1'),
        
        # Saída binária (1 neurônio com sigmoid para classificação binária)
        layers.Dense(1, activation='sigmoid', name='output')
    ])
    
    # Compilação do modelo
    # Explicação da loss function:
    # - binary_crossentropy: Apropriada para classificação binária
    # - Mede a diferença entre probabilidades preditas e labels verdadeiros
    #
    # Explicação do otimizador:
    # - Adam: Otimizador adaptativo, funciona bem na maioria dos casos
    # - Combina vantagens de RMSprop e SGD com momentum
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall']
    )
    
    return model

# Cria e exibe sumário do modelo
model = create_simple_cnn_model()
model.summary()

In [ ]:
def load_simulation_data(simulation_num):
    """
    Carrega dados de uma simulação específica.
    
    O dataset está organizado em:
    datasets/
      sim01/
        raiva/     [50 imagens]
        alegria/   [50 imagens]
      sim02/
        ...
    
    Keras image_dataset_from_directory automaticamente:
    - Carrega imagens dos subdiretórios
    - Cria labels baseado nos nomes das pastas
    - Redimensiona imagens para o tamanho especificado
    - Divide em treino/validação conforme validation_split
    
    Args:
        simulation_num: Número da simulação (1-30)
    
    Returns:
        train_ds, val_ds: Datasets de treino e validação
    """
    sim_dir = DATASET_DIR / f"sim{simulation_num:02d}"
    
    if not sim_dir.exists():
        raise FileNotFoundError(f"Simulação {simulation_num} não encontrada em {sim_dir}")
    
    # Dataset de treino
    # Explicação do validation_split:
    # - 0.2 significa 20% dos dados para validação
    # - subset="training" seleciona os 80% para treino
    train_ds = keras.utils.image_dataset_from_directory(
        sim_dir,
        validation_split=VALIDATION_SPLIT,
        subset="training",
        seed=SEED,
        image_size=(IMG_WIDTH, IMG_HEIGHT),
        batch_size=BATCH_SIZE,
        label_mode='binary'  # Classificação binária: 0 ou 1
    )
    
    # Dataset de validação
    val_ds = keras.utils.image_dataset_from_directory(
        sim_dir,
        validation_split=VALIDATION_SPLIT,
        subset="validation",
        seed=SEED,
        image_size=(IMG_WIDTH, IMG_HEIGHT),
        batch_size=BATCH_SIZE,
        label_mode='binary'
    )
    
    # Otimizações de performance
    # Explicação:
    # - cache(): Mantém imagens em memória após primeira época (acelera treinamento)
    # - shuffle(1000): Embaralha dados para evitar viés de ordem
    # - prefetch(): Prepara próximo batch enquanto treina o atual (paralelização)
    train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
    val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    
    return train_ds, val_ds

# Teste de carregamento
print("Testando carregamento de dados da simulação 01...")
train_ds, val_ds = load_simulation_data(1)
print(f"Dataset de treino: {train_ds}")
print(f"Dataset de validação: {val_ds}")

In [ ]:
def train_simulation(simulation_num, verbose=1):
    """
    Treina o modelo em uma simulação específica.
    
    Pipeline de treinamento:
    1. Carrega dados da simulação
    2. Cria novo modelo (inicialização aleatória)
    3. Configura callbacks (early stopping, model checkpoint)
    4. Treina modelo
    5. Salva modelo e histórico
    6. Retorna métricas finais
    
    Args:
        simulation_num: Número da simulação (1-30)
        verbose: Nível de verbosidade (0=silencioso, 1=barra de progresso, 2=uma linha por época)
    
    Returns:
        dict com métricas da simulação
    """
    print(f"\n{'='*70}")
    print(f"Iniciando Simulação {simulation_num:02d}/{NUM_SIMULATIONS}")
    print(f"{'='*70}")
    
    # 1. Carrega dados
    train_ds, val_ds = load_simulation_data(simulation_num)
    
    # 2. Cria novo modelo
    # Importante: Cada simulação usa um modelo com pesos iniciais diferentes
    model = create_simple_cnn_model()
    
    # 3. Configura callbacks
    # Early Stopping: Para treinamento se validação não melhorar por N épocas
    # Explicação:
    # - monitor='val_loss': Monitora loss de validação
    # - patience=5: Espera 5 épocas sem melhoria antes de parar
    # - restore_best_weights=True: Restaura pesos da melhor época
    early_stop = keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    )
    
    # Model Checkpoint: Salva melhor modelo durante treinamento
    model_path = MODELS_DIR / f"sim{simulation_num:02d}_best.keras"
    checkpoint = keras.callbacks.ModelCheckpoint(
        model_path,
        monitor='val_accuracy',
        save_best_only=True,
        verbose=0
    )
    
    # 4. Treina modelo
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS,
        callbacks=[early_stop, checkpoint],
        verbose=verbose
    )
    
    # 5. Salva histórico de treinamento
    history_path = RESULTS_DIR / f"sim{simulation_num:02d}_history.json"
    with open(history_path, 'w') as f:
        # Converte arrays numpy para listas para serialização JSON
        history_dict = {key: [float(val) for val in values] 
                       for key, values in history.history.items()}
        json.dump(history_dict, f, indent=2)
    
    # 6. Coleta métricas finais
    # Usa métricas da última época (com best weights restaurados)
    final_metrics = {
        'simulation': simulation_num,
        'train_accuracy': float(history.history['accuracy'][-1]),
        'val_accuracy': float(history.history['val_accuracy'][-1]),
        'train_loss': float(history.history['loss'][-1]),
        'val_loss': float(history.history['val_loss'][-1]),
        'epochs_trained': len(history.history['loss']),
        'model_path': str(model_path),
        'timestamp': datetime.now().isoformat()
    }
    
    print(f"\nSimulação {simulation_num:02d} concluída:")
    print(f"  Acurácia Treino: {final_metrics['train_accuracy']:.4f}")
    print(f"  Acurácia Validação: {final_metrics['val_accuracy']:.4f}")
    print(f"  Épocas: {final_metrics['epochs_trained']}")
    
    return final_metrics

# Teste de treinamento em uma simulação
print("Executando treinamento de teste na simulação 01...")
# Descomente a linha abaixo para executar teste
# test_metrics = train_simulation(1, verbose=1)

In [ ]:
def run_all_simulations():
    """
    Executa treinamento em todas as 30 simulações.
    
    Este processo pode levar várias horas dependendo do hardware:
    - Com GPU: ~5-10 min por simulação = 2.5-5 horas total
    - Sem GPU: ~15-30 min por simulação = 7.5-15 horas total
    
    Salva resultados consolidados em CSV para análise posterior.
    
    Returns:
        DataFrame com métricas de todas as simulações
    """
    all_results = []
    
    print("\n" + "="*70)
    print("INICIANDO TREINAMENTO DE TODAS AS SIMULAÇÕES")
    print("="*70)
    print(f"Total de simulações: {NUM_SIMULATIONS}")
    print(f"Epochs máximos por simulação: {EPOCHS}")
    print(f"Early stopping: {5} épocas de paciência")
    print("="*70 + "\n")
    
    start_time = datetime.now()
    
    for sim_num in range(1, NUM_SIMULATIONS + 1):
        try:
            # Treina simulação
            metrics = train_simulation(sim_num, verbose=1)
            all_results.append(metrics)
            
            # Salva resultados parciais (segurança em caso de interrupção)
            partial_df = pd.DataFrame(all_results)
            partial_df.to_csv(RESULTS_DIR / "partial_results.csv", index=False)
            
        except Exception as e:
            print(f"\nERRO na simulação {sim_num}: {e}")
            print("Continuando com próxima simulação...\n")
            continue
    
    end_time = datetime.now()
    duration = end_time - start_time
    
    # Converte para DataFrame
    results_df = pd.DataFrame(all_results)
    
    # Salva resultados finais
    results_path = RESULTS_DIR / "all_simulations_results.csv"
    results_df.to_csv(results_path, index=False)
    
    print("\n" + "="*70)
    print("TODAS AS SIMULAÇÕES CONCLUÍDAS")
    print("="*70)
    print(f"Tempo total: {duration}")
    print(f"Tempo médio por simulação: {duration / len(all_results)}")
    print(f"Resultados salvos em: {results_path}")
    print("="*70)
    
    return results_df

# EXECUÇÃO: Descomente a linha abaixo para executar todas as simulações
# ATENÇÃO: Este processo pode levar várias horas!
# results_df = run_all_simulations()

In [ ]:
def analyze_results(results_df):
    """
    Analisa resultados das simulações e gera estatísticas.
    
    Calcula:
    - Média e desvio padrão de acurácia
    - Intervalo de confiança (95%)
    - Melhor e pior simulação
    - Distribuição de épocas de convergência
    
    Args:
        results_df: DataFrame com resultados das simulações
    
    Returns:
        dict com estatísticas agregadas
    """
    stats = {
        # Acurácia de validação (principal métrica)
        'val_accuracy_mean': results_df['val_accuracy'].mean(),
        'val_accuracy_std': results_df['val_accuracy'].std(),
        'val_accuracy_min': results_df['val_accuracy'].min(),
        'val_accuracy_max': results_df['val_accuracy'].max(),
        
        # Intervalo de confiança 95% (assumindo distribuição normal)
        # IC = média ± 1.96 * (desvio_padrão / sqrt(n))
        'val_accuracy_ci_lower': results_df['val_accuracy'].mean() - 
                                 1.96 * results_df['val_accuracy'].std() / np.sqrt(len(results_df)),
        'val_accuracy_ci_upper': results_df['val_accuracy'].mean() + 
                                 1.96 * results_df['val_accuracy'].std() / np.sqrt(len(results_df)),
        
        # Épocas de treinamento
        'epochs_mean': results_df['epochs_trained'].mean(),
        'epochs_std': results_df['epochs_trained'].std(),
        
        # Loss de validação
        'val_loss_mean': results_df['val_loss'].mean(),
        'val_loss_std': results_df['val_loss'].std(),
        
        # Número de simulações
        'num_simulations': len(results_df)
    }
    
    # Identifica melhor e pior simulação
    best_idx = results_df['val_accuracy'].idxmax()
    worst_idx = results_df['val_accuracy'].idxmin()
    
    stats['best_simulation'] = int(results_df.loc[best_idx, 'simulation'])
    stats['best_accuracy'] = float(results_df.loc[best_idx, 'val_accuracy'])
    stats['worst_simulation'] = int(results_df.loc[worst_idx, 'simulation'])
    stats['worst_accuracy'] = float(results_df.loc[worst_idx, 'val_accuracy'])
    
    # Exibe resumo
    print("\n" + "="*70)
    print("ANÁLISE ESTATÍSTICA DOS RESULTADOS")
    print("="*70)
    print(f"\nSimulações analisadas: {stats['num_simulations']}")
    print(f"\nAcurácia de Validação:")
    print(f"  Média: {stats['val_accuracy_mean']:.4f} ± {stats['val_accuracy_std']:.4f}")
    print(f"  IC 95%: [{stats['val_accuracy_ci_lower']:.4f}, {stats['val_accuracy_ci_upper']:.4f}]")
    print(f"  Min: {stats['val_accuracy_min']:.4f} (sim {stats['worst_simulation']:02d})")
    print(f"  Max: {stats['val_accuracy_max']:.4f} (sim {stats['best_simulation']:02d})")
    print(f"\nÉpocas de Treinamento:")
    print(f"  Média: {stats['epochs_mean']:.1f} ± {stats['epochs_std']:.1f}")
    print(f"\nLoss de Validação:")
    print(f"  Média: {stats['val_loss_mean']:.4f} ± {stats['val_loss_std']:.4f}")
    print("="*70)
    
    # Salva estatísticas
    stats_path = RESULTS_DIR / "statistical_summary.json"
    with open(stats_path, 'w') as f:
        json.dump(stats, f, indent=2)
    print(f"\nEstatísticas salvas em: {stats_path}")
    
    return stats

# Exemplo de uso (após executar simulações)
# stats = analyze_results(results_df)

In [ ]:
def plot_results(results_df):
    """
    Cria visualizações dos resultados das simulações.
    
    Gera 4 gráficos:
    1. Distribuição de acurácias (histograma)
    2. Acurácia por simulação (linha)
    3. Treino vs Validação (scatter)
    4. Épocas de convergência (box plot)
    
    Args:
        results_df: DataFrame com resultados das simulações
    """
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. Distribuição de acurácias
    ax = axes[0, 0]
    ax.hist(results_df['val_accuracy'], bins=15, edgecolor='black', alpha=0.7)
    ax.axvline(results_df['val_accuracy'].mean(), color='red', 
               linestyle='--', linewidth=2, label='Média')
    ax.set_xlabel('Acurácia de Validação')
    ax.set_ylabel('Frequência')
    ax.set_title('Distribuição de Acurácias (30 Simulações)')
    ax.legend()
    ax.grid(alpha=0.3)
    
    # 2. Acurácia por simulação
    ax = axes[0, 1]
    ax.plot(results_df['simulation'], results_df['val_accuracy'], 
            marker='o', linewidth=1, markersize=4)
    ax.axhline(results_df['val_accuracy'].mean(), color='red', 
               linestyle='--', linewidth=2, label='Média')
    ax.fill_between(results_df['simulation'],
                     results_df['val_accuracy'].mean() - results_df['val_accuracy'].std(),
                     results_df['val_accuracy'].mean() + results_df['val_accuracy'].std(),
                     alpha=0.2, color='red', label='±1 Desvio Padrão')
    ax.set_xlabel('Número da Simulação')
    ax.set_ylabel('Acurácia de Validação')
    ax.set_title('Acurácia por Simulação')
    ax.legend()
    ax.grid(alpha=0.3)
    
    # 3. Treino vs Validação
    ax = axes[1, 0]
    ax.scatter(results_df['train_accuracy'], results_df['val_accuracy'], 
               alpha=0.6, s=50)
    # Linha de referência (treino = validação)
    min_val = min(results_df['train_accuracy'].min(), results_df['val_accuracy'].min())
    max_val = max(results_df['train_accuracy'].max(), results_df['val_accuracy'].max())
    ax.plot([min_val, max_val], [min_val, max_val], 
            'r--', linewidth=2, label='Linha Ideal')
    ax.set_xlabel('Acurácia de Treino')
    ax.set_ylabel('Acurácia de Validação')
    ax.set_title('Treino vs Validação (Detecção de Overfitting)')
    ax.legend()
    ax.grid(alpha=0.3)
    
    # 4. Épocas de convergência
    ax = axes[1, 1]
    bp = ax.boxplot([results_df['epochs_trained']], 
                     labels=['Épocas'], patch_artist=True)
    bp['boxes'][0].set_facecolor('lightblue')
    ax.set_ylabel('Número de Épocas')
    ax.set_title('Distribuição de Épocas de Convergência')
    ax.grid(alpha=0.3, axis='y')
    
    plt.tight_layout()
    
    # Salva figura
    fig_path = RESULTS_DIR / "results_visualization.png"
    plt.savefig(fig_path, dpi=300, bbox_inches='tight')
    print(f"Visualização salva em: {fig_path}")
    
    plt.show()

# Exemplo de uso
# plot_results(results_df)

In [ ]:
# Célula para carregar e analisar resultados existentes
# Útil se você já executou as simulações e quer apenas ver os resultados

def load_existing_results():
    """
    Carrega resultados de simulações já executadas.
    
    Returns:
        DataFrame com resultados ou None se não existir
    """
    results_path = RESULTS_DIR / "all_simulations_results.csv"
    
    if not results_path.exists():
        print(f"Arquivo de resultados não encontrado: {results_path}")
        print("Execute run_all_simulations() primeiro.")
        return None
    
    df = pd.read_csv(results_path)
    print(f"Resultados carregados: {len(df)} simulações")
    return df

# Carrega resultados existentes
# results_df = load_existing_results()

# Se existir, analisa e visualiza
# if results_df is not None:
#     stats = analyze_results(results_df)
#     plot_results(results_df)

## Próximos Passos

Após executar todas as simulações e analisar os resultados:

1. **Comparação com Foundation Models**: Compare os resultados desta CNN com modelos CLIP ou outros foundation models

2. **Análise de Erros**: Investigue quais imagens foram classificadas incorretamente

3. **Interpretabilidade**: Use técnicas como Grad-CAM para visualizar quais regiões da imagem a CNN usa para classificação

4. **Otimização**: Experimente diferentes arquiteturas, hiperparâmetros ou técnicas de data augmentation

5. **Documentação**: Documente achados e prepare relatório comparativo

## Conceitos Aprendidos

- Arquitetura de CNNs para classificação de imagens
- Importância de múltiplas simulações para avaliação robusta
- Técnicas de regularização (Dropout, Early Stopping)
- Análise estatística de resultados experimentais
- Pipeline completo de ML: dados → treino → validação → análise